In [1]:
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# https://web.archive.org/web/20090722233436/http://blog.brandonpelfrey.com/?p=303

class Neighbor:
    def __init__(self, i, j, q, q2): # q and q2 are weighted distances 
        # int
        self.i = i 
        self.j = j
        # float
        self.q = q
        self.q2 = q2

    

class Particle:
    def __init__(self, pos, mass, rho, sigma, beta): # sigma and beta controls the viscosity of the flow
        # vec2
        self.pos = pos
        self.pos_old = np.zeros(2)
        self.vel = np.zeros(2)
        self.force = np.zeros(2)
        # float
        self.mass = mass
        self.rho = rho
        self.rho_near = 0
        self.press = 0
        self.press_near = 0
        self.sigma = sigma
        self.beta = beta
        # list
        self.neighbors = []

    def add_neighbor(self, neighbor):
        self.neighbors.append(neighbor)



In [4]:
WIDTH = HEIGHT = 32
N = 10 # Number of Particles in the simulation

G = 0.02 * 0.25 # Gravitational Constant for our simulation
spacing = 1 # Spacing of particles
k = spacing / 1000 # Far pressure weight
k_near = k*10 # Near pressure weight
rest_density = 3 # Rest Density
r=spacing*1.25 # Radius of Support
rsq=r*r # squared for performance

SIM_W = 50 # size of the world
bottom = 0 # The floor of the world

mass = 1
rho = 1
sigma = 1
beta = 1

particle_list = []
for x in range(1, WIDTH, spacing):
    for y in range(1, HEIGHT, spacing):
        pos = np.array([x,y])
        particle = Particle(pos, mass, rho, sigma, beta)
        particle_list.append(particle)


In [5]:
# Density: to know the density of “stuff” around a particle
for i, particle_i in enumerate(particle_list):
    d = 0
    dn = 0
    for j, particle_j in enumerate(particle_list):
        # By doing this test, we insure there is only one computation per pair of particles
        # And that there is no computation between a particle and itself
        if (i >= j): continue 

        rij = particle_j.pos - particle_i.pos
        dist2 = np.linalg.norm(rij)**2 # square distance
        if dist2 < rsq:
            length = np.sqrt(dist2)

            q = 1 - length / r
            q2 = q**2
            q3 = q**3
            
            d += q2
            dn += q3
            particle_j.rho += q2
            particle_j.rho_near += q3

            neighbor = Neighbor(i, j, q, q2)
            particle_i.add_neighbor(neighbor)

    particle_i.rho += d
    particle_i.rho_near += dn
    

In [9]:
a = np.ones(2)
print(a)

magnitude = np.linalg.norm(a)

# Get the unit vector
unit_vector = a / magnitude
unit_vector

[1. 1.]


array([0.70710678, 0.70710678])

In [17]:
# Pressure Calculation
for i, particle_i in enumerate(particle_list):
    particle_i.press = k * (particle_i.rho - rest_density)
    particle_i.press_near = k_near * particle_i.rho_near

# Pressure Force Calculation:
# We will force particles in or out from their neighbors based on their difference from the rest density.
for i, particle_i in enumerate(particle_list):
    dx = np.zeros(2)
    # ncount = len(particle_i.neighbors)
    for neighbor in particle_i.neighbors:
        j = neighbor.j
        rij = rij = particle_j.pos - particle_i.pos
        dm = (particle_i.press + particle_j.press) * q + \
            (particle_i.press_near + particle_j.press_near) * q2 # magnitude of the force
        direction = rij / np.linalg.norm(rij) # unit direction of the force
        D = direction * dm 
        dx += D 
        particle_j.force += D

    particle_i.force -= dx
        



In [ ]:
# Viscous Force Calculation (later)
